In [1]:
import cv2
import torch
from ultralytics import YOLO

In [2]:
model = YOLO('yolo11n.pt')

In [ ]:
video_path = 'carsonroad.mp4'

capture = cv2.VideoCapture(video_path)

width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(capture.get(cv2.CAP_PROP_FPS))
output_path = 'output_mot.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

y_line = int(height/2)
car_counter = 0
car_ids_crossed = set()

while capture.isOpened():
    ret, frame = capture.read()

    if not ret:
        break

    results = model.track(frame, persist = True)

    for det in results[0].boxes:
        # result[0].boxes have cls(tensor), conf(tensor), data(tensor), id, shape(tensor), xywh(tensor), xyxy(tensor)
        class_id = int(det.cls) # 0: person, 2: car ....
        track_id = None

        if det.id:
            track_id = int(det.id)

        x1, y1, x2, y2 = map(int, det.xyxy[0])

        # class_id 2 is typically used for car 
        if class_id == 2:
            if track_id and track_id not in car_ids_crossed and y1 < y_line < y2:
                car_counter+=1
                car_ids_crossed.add(track_id)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.line(frame, (0, y_line), (width, y_line), (0, 255, 0), 2)
    cv2.putText(frame, f"Cars Counted: {car_counter}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('YOLOv11 Car Counting', frame)
    output.write(frame)

    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

capture.release()
output.release()
cv2.destroyAllWindows()